In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
USE_CUDA = torch.cuda.is_available()

from data import load_squad_data,preprop,getBatch,pad_to_batch
from model import CoattentionEncoder, DynamicDecoder

In [ ]:
MAX_LEN=600

In [2]:
word2index,index2word,train_data = preprop(load_squad_data('train-v1.1.json',MAX_LEN))

Skipped 763, 86817 question/answer
Successfully Build 115268 vocabs
Preprop Complete!


In [6]:
EMBED_SIE=200
HIDDEN_SIZE=200
MAXOUT_POOL=4
MAX_ITER=4
BATCH_SIZE=16
STEP=100
LR=0.001
encoder = CoattentionEncoder(len(word2index),EMBED_SIE,HIDDEN_SIZE,use_cuda=USE_CUDA)
decoder = DynamicDecoder(HIDDEN_SIZE,MAXOUT_POOL,max_iter=MAX_ITER,use_cuda=USE_CUDA)

loss_function = nn.CrossEntropyLoss()
enc_optim = optim.Adam(encoder.parameters(),lr=LR)
dec_optim = optim.Adam(decoder.parameters(),lr=LR)

In [7]:
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        documents,questions,starts,ends = pad_to_batch(batch,word2index)

        encoder.zero_grad()
        decoder.zero_grad()
        U = encoder(documents,questions,True)
        _,_,entropies = decoder(U,True)

        s_ents, e_ents = list(zip(*entropies)) # x MAX_ITER
        loss_start,loss_end=0,0
        for m in range(MAX_ITER):
            loss_start+=loss_function(s_ents[m],starts.view(-1))
            loss_end+=loss_function(s_ents[m],ends.view(-1))

        loss = loss_start+loss_end
        losses.append(loss.data[0])
        loss.backward()
        torch.nn.utils.clip_grad_norm(encoder.parameters(), 10) # gradient clipping
        torch.nn.utils.clip_grad_norm(decoder.parameters(), 10) 
        enc_optim.step()
        dec_optim.step()
        
        if i % 100 == 0:
            print("[%d/%d] [%d/%d] loss : %.3f" % (step,STEP,i,len(train_data)//BATCH_SIZE,np.mean(losses)))
            losses=[]

[0/100] [0/5426] loss : 42.041


KeyboardInterrupt: 